Accessing as AbhayBisht0801

Initialized MLflow to track repo "AbhayBisht0801/Loan-Defaulter-Prediction"

Repository AbhayBisht0801/Loan-Defaulter-Prediction initialized!

In [2]:
import os
os.chdir('../')


In [3]:
from dataclasses import dataclass
from pathlib import Path
@dataclass
class MLFlowTrackingConfig:
    root_dir:Path
    test_data_path:Path
    best_model:Path
    metrics_file_name:Path
    mlflow_uri:str

In [4]:
from src.Loan_defaulter.constants import *
from src.Loan_defaulter.utils.common import read_yaml,create_directories

In [5]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath=CONFIG_FILE_PATH,
            params_filepath=PARAMS_FILE_PATH

    ):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        

    def get_model_tracking_config(self)->MLFlowTrackingConfig:
        config=self.config
       
        create_directories([config.mlflow.root_dir])

        model_tracking_config=MLFlowTrackingConfig(
            root_dir=config.mlflow.root_dir,
            test_data_path=config.data_transformation.split_dir,
            best_model=config.model_training.best_model,
            metrics_file_name=config.mlflow.metrics_file_name,
            mlflow_uri="https://dagshub.com/AbhayBisht0801/Loan-Defaulter-Prediction.mlflow"
        )
        return model_tracking_config

In [6]:
import os
import pandas as pd
from sklearn.metrics import accuracy_score,recall_score,f1_score,precision_score
import mlflow
import mlflow.sklearn
import numpy as np
import pickle
import joblib 
from src.Loan_defaulter.utils.common import save_json
from urllib.parse import urlparse
import dagshub

In [21]:
class MLFLOWTracking:
    def __init__(self,config=MLFlowTrackingConfig):
        self.config = config
    def evaL_metrics(self,actual,pred):
        model_precision=precision_score(actual,pred)
        model_recall=recall_score(actual,pred)
        model_accuracy=accuracy_score(actual,pred)
        model_f1_score=accuracy_score(actual,pred)
        return {'accuracy':model_accuracy,'f1_score':model_f1_score,'precision':model_precision,'recall':model_recall}
    def log_into_mlflow(self):
        
        dagshub.init(repo_owner='AbhayBisht0801', repo_name='Loan-Defaulter-Prediction', mlflow=True)
        val_data=pickle.load(open(os.path.join(self.config.test_data_path,'val.pkl'),'rb'))
        X_val=val_data[:,:-1]
        y_val=val_data[:,-1]
        
        model=pickle.load(open(self.config.best_model,'rb'))
        with mlflow.start_run():
            pred=model.predict(X_val)
            metrics=self.evaL_metrics(y_val,pred)
            print(metrics)
            mlflow.log_params(model.get_params())
            save_json(path=Path(self.config.metrics_file_name),data=metrics)
            mlflow.log_metric('accuracy',metrics['accuracy'])
            mlflow.log_metric('f1_score',metrics['f1_score'])
            mlflow.log_metric('precision',metrics['precision'])
            mlflow.log_metric('recall',metrics['recall'])
            
            mlflow.sklearn.log_model(model,"model",registered_model_name="KNN")


        
        



In [22]:
try:
    config=ConfigurationManager()
    mlflow_tracking_config=config.get_model_tracking_config()
    mlflow_tracking=MLFLOWTracking(config=mlflow_tracking_config)
    mlflow_tracking.log_into_mlflow()
except Exception as e:
    raise e
    

[2024-12-29 12:45:09,954:INFO:common:yaml file: config\config.yaml loaded successfully]
[2024-12-29 12:45:09,956:INFO:common:yaml file: params.yaml loaded successfully]
[2024-12-29 12:45:09,958:INFO:common:created directory at: artifacts/mlflow]
{'accuracy': 0.8970403587443946, 'f1_score': 0.8970403587443946, 'precision': 0.719661205220772, 'recall': 0.9492673992673992}
[2024-12-29 12:45:26,834:INFO:common:json file saved at: artifacts\mlflow\metrics.json]


c:\Users\bisht\OneDrive\Desktop\New folder\venv\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Successfully registered model 'KNN'.
2024/12/29 12:49:00 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: KNN, version 1
Created version '1' of model 'KNN'.


{'bootstrap': True,
 'bootstrap_features': False,
 'estimator__algorithm': 'auto',
 'estimator__leaf_size': 30,
 'estimator__metric': 'minkowski',
 'estimator__metric_params': None,
 'estimator__n_jobs': -1,
 'estimator__n_neighbors': 5,
 'estimator__p': 2,
 'estimator__weights': 'uniform',
 'estimator': KNeighborsClassifier(n_jobs=-1),
 'max_features': 1.0,
 'max_samples': 1.0,
 'n_estimators': 10,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 42,
 'replacement': False,
 'sampler': None,
 'sampling_strategy': 'auto',
 'verbose': 0,
 'warm_start': False}

In [14]:
X_val=pickle.load(open('artifacts\data_transformation\split_data\\val.pkl','rb'))
X_val[:,-1]

array([1., 0., 0., ..., 0., 0., 1.])